# Фитирование синтетических кривых рассеяния

Фитирование (аналитическое аппроксимирование) всех кривых малоуглового рассеяния нейтронов моделью сферических оболочек будет производиться с помощью утилит программы Sasview, реализованных на языке Python.

In [1]:
import glob
import pandas as pd
import os
from bumps.names import *
from sasmodels.core import load_model
from sasmodels.bumps_model import Model, Experiment
from sasmodels.data import load_data
from bumps.fitters import fit

Напишем функцию, с помощью которой будет осуществляться фитирование синтетических кривых малоуглового рассеяния нейтронов в батч-режиме для определения количественных значений целевого признака thickness. В функции будет указана модель для фитирования (модель сферических оболочек), необходимые параметры этой модели, какие параметры будут фиксированными, а какие - свободными, а также диапазон допустимых значений этих параметров. Поскольку все файлы отсортированы от максимального радиуса везикул до минимального, а настройки параметров модели в процессе фитирования каждой кривой зависят от предыдущего шага (предыдущей кривой), фитирование будет осуществляться следующим образом: для каждого батча определенной полидисперсности (1000 кривых рассеяния) начальным значением радиуса везикул будет максимальный радиус из возможных; для каждой последующей кривой этот параметр будет постепенно уменьшаться с определенным заданным шагом. Параметры масштабирования (scale), фона (background), радиуса везикул (radius), толщины мембраны (thickness) будут свободными (фитируемыми), а остальные параметры в модели сферических оболочек - фиксированными (например, параметр полидисперсности по размерам) (https://www.sasview.org/docs/user/models/vesicle.html). Вдобавок, значения фитируемых параметров будут фиксированы в определенных физически осмысленных диапазонах во избежание расходимостей при оптимизации параметров.

In [2]:
def batch_fit(file_name_in, file_name_out, koeff=0.9985, radius_pd=None, fixed_backgrounds=None):
    
    data_files = glob.glob(file_name_in)
    results = []
    current_radius = 430

    for i, file_path in enumerate(data_files):
        filename = os.path.basename(file_path)
        print(f"[{i+1}/{len(data_files)}] {filename}")

        data = load_data(file_path)

        if i > 0:
            current_radius = max(90, current_radius * koeff)

        # Определяем background: фиксированный или обычный
        if fixed_backgrounds and filename in fixed_backgrounds:
            background_value = fixed_backgrounds[filename]
            background_fixed = True
        else:
            background_value = 0.05
            background_fixed = False

        model = Model(
            model=load_model('vesicle'),
            scale=0.45,
            background=background_value,
            radius=current_radius,
            thickness=40.0,
            radius_pd=radius_pd,
            thickness_pd=0.0
        )
        
        # Диапазоны для свободных параметров
        model.radius.range(80, 450)
        model.thickness.range(25, 55)
        model.scale.range(0.05, 2)
        
        # Диапазон для background, только если не фиксирован
        if not background_fixed:
            model.background.range(0.005, 0.2)
        else:
            model.background.fixed = True 

        model.radius_pd.fixed = True
        model.thickness_pd.fixed = True

        experiment = Experiment(data=data, model=model)
        problem = FitProblem(experiment)
        fit(problem, method='lm', steps=100, verbose=False)

        current_radius = model.radius.value

        results.append({
            'filename': filename,
            'radius_fitted': model.radius.value,
            'thickness_fitted': model.thickness.value,
            'scale': model.scale.value,
            'background': model.background.value,
            'chi2': problem.chisq()
        })

        bg_status = "fixed" if background_fixed else "fitted"
        print(f" radius: {model.radius.value:.1f}A, thickness: {model.thickness.value:.1f}A,\
        background: {model.background.value:.3f} ({bg_status}), chi2: {problem.chisq():.2f}")


    df = pd.DataFrame(results)
    df.to_csv(file_name_out, index=False)


Каждый вызов функции - для набора файлов с кривыми, соответствующими определенной полидисперсности везикул по радиусам (с параметром полидисперсности от 0.20 до 0.40), и с определенным коэффициентом, линейным образом уменьшающим изначально заданное значение радиуса. При фитировании хи-квадрат должен стремиться в единице; идеальный фит - с хи-квадрат равным единице.

In [3]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_20/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_20.csv',
          koeff = 0.9975, radius_pd=0.24)

[1/1000] vesicle_0000.dat
 radius: 418.7A, thickness: 49.2A,        background: 0.027 (fitted), chi2: 2.02
[2/1000] vesicle_0001.dat
 radius: 420.0A, thickness: 39.9A,        background: 0.023 (fitted), chi2: 1.28
[3/1000] vesicle_0002.dat
 radius: 420.1A, thickness: 39.9A,        background: 0.005 (fitted), chi2: 3.39
[4/1000] vesicle_0003.dat
 radius: 416.1A, thickness: 41.5A,        background: 0.030 (fitted), chi2: 1.04
[5/1000] vesicle_0004.dat
 radius: 411.9A, thickness: 47.7A,        background: 0.020 (fitted), chi2: 2.58
[6/1000] vesicle_0005.dat
 radius: 406.1A, thickness: 32.0A,        background: 0.021 (fitted), chi2: 1.43
[7/1000] vesicle_0006.dat
 radius: 411.1A, thickness: 49.2A,        background: 0.025 (fitted), chi2: 1.01
[8/1000] vesicle_0007.dat
 radius: 397.7A, thickness: 43.0A,        background: 0.034 (fitted), chi2: 1.40
[9/1000] vesicle_0008.dat
 radius: 395.9A, thickness: 45.8A,        background: 0.036 (fitted), chi2: 1.50
[10/1000] vesicle_0009.dat
 radius: 4

In [4]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_21/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_21.csv',
          koeff = 0.9975, radius_pd=0.25)

[1/1000] vesicle_0000.dat
 radius: 447.7A, thickness: 44.1A,        background: 0.017 (fitted), chi2: 2.97
[2/1000] vesicle_0001.dat
 radius: 421.2A, thickness: 35.1A,        background: 0.016 (fitted), chi2: 1.76
[3/1000] vesicle_0002.dat
 radius: 385.3A, thickness: 48.6A,        background: 0.018 (fitted), chi2: 1.06
[4/1000] vesicle_0003.dat
 radius: 381.6A, thickness: 41.9A,        background: 0.021 (fitted), chi2: 1.16
[5/1000] vesicle_0004.dat
 radius: 379.9A, thickness: 45.8A,        background: 0.024 (fitted), chi2: 1.55
[6/1000] vesicle_0005.dat
 radius: 387.1A, thickness: 44.6A,        background: 0.020 (fitted), chi2: 1.17
[7/1000] vesicle_0006.dat
 radius: 352.7A, thickness: 37.6A,        background: 0.020 (fitted), chi2: 1.22
[8/1000] vesicle_0007.dat
 radius: 347.8A, thickness: 49.1A,        background: 0.030 (fitted), chi2: 1.30
[9/1000] vesicle_0008.dat
 radius: 340.1A, thickness: 46.3A,        background: 0.034 (fitted), chi2: 0.61
[10/1000] vesicle_0009.dat
 radius: 3

In [5]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_22/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_22.csv',
          koeff = 0.9975, 
          radius_pd=0.25)

[1/1000] vesicle_0000.dat
 radius: 450.0A, thickness: 36.1A,        background: 0.043 (fitted), chi2: 1.32
[2/1000] vesicle_0001.dat
 radius: 435.6A, thickness: 41.2A,        background: 0.031 (fitted), chi2: 2.30
[3/1000] vesicle_0002.dat
 radius: 450.0A, thickness: 42.3A,        background: 0.023 (fitted), chi2: 2.41
[4/1000] vesicle_0003.dat
 radius: 435.6A, thickness: 42.8A,        background: 0.036 (fitted), chi2: 1.20
[5/1000] vesicle_0004.dat
 radius: 406.9A, thickness: 38.7A,        background: 0.025 (fitted), chi2: 1.09
[6/1000] vesicle_0005.dat
 radius: 395.5A, thickness: 47.0A,        background: 0.025 (fitted), chi2: 1.12
[7/1000] vesicle_0006.dat
 radius: 356.2A, thickness: 32.1A,        background: 0.036 (fitted), chi2: 0.77
[8/1000] vesicle_0007.dat
 radius: 360.3A, thickness: 32.2A,        background: 0.028 (fitted), chi2: 0.35
[9/1000] vesicle_0008.dat
 radius: 355.6A, thickness: 43.7A,        background: 0.022 (fitted), chi2: 1.30
[10/1000] vesicle_0009.dat
 radius: 3

In [6]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_23/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_23.csv',
          koeff = 0.9975, 
          radius_pd=0.26)

[1/1000] vesicle_0000.dat
 radius: 450.0A, thickness: 41.8A,        background: 0.036 (fitted), chi2: 1.31
[2/1000] vesicle_0001.dat
 radius: 450.0A, thickness: 34.3A,        background: 0.024 (fitted), chi2: 1.36
[3/1000] vesicle_0002.dat
 radius: 450.0A, thickness: 34.4A,        background: 0.029 (fitted), chi2: 1.47
[4/1000] vesicle_0003.dat
 radius: 450.0A, thickness: 31.6A,        background: 0.027 (fitted), chi2: 1.23
[5/1000] vesicle_0004.dat
 radius: 450.0A, thickness: 29.3A,        background: 0.016 (fitted), chi2: 6.89
[6/1000] vesicle_0005.dat
 radius: 417.7A, thickness: 41.5A,        background: 0.025 (fitted), chi2: 1.15
[7/1000] vesicle_0006.dat
 radius: 380.4A, thickness: 41.2A,        background: 0.034 (fitted), chi2: 0.81
[8/1000] vesicle_0007.dat
 radius: 355.6A, thickness: 36.4A,        background: 0.026 (fitted), chi2: 1.33
[9/1000] vesicle_0008.dat
 radius: 357.2A, thickness: 39.0A,        background: 0.028 (fitted), chi2: 1.34
[10/1000] vesicle_0009.dat
 radius: 3

In [7]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_24/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_24.csv',
          koeff = 0.9975, 
          radius_pd=0.27)

[1/1000] vesicle_0000.dat
 radius: 450.0A, thickness: 32.3A,        background: 0.028 (fitted), chi2: 0.99
[2/1000] vesicle_0001.dat
 radius: 448.3A, thickness: 38.9A,        background: 0.042 (fitted), chi2: 0.74
[3/1000] vesicle_0002.dat
 radius: 447.2A, thickness: 48.3A,        background: 0.021 (fitted), chi2: 1.40
[4/1000] vesicle_0003.dat
 radius: 450.0A, thickness: 35.3A,        background: 0.029 (fitted), chi2: 0.66
[5/1000] vesicle_0004.dat
 radius: 450.0A, thickness: 30.4A,        background: 0.035 (fitted), chi2: 1.21
[6/1000] vesicle_0005.dat
 radius: 450.0A, thickness: 43.0A,        background: 0.026 (fitted), chi2: 1.01
[7/1000] vesicle_0006.dat
 radius: 446.3A, thickness: 40.3A,        background: 0.026 (fitted), chi2: 1.01
[8/1000] vesicle_0007.dat
 radius: 450.0A, thickness: 33.3A,        background: 0.026 (fitted), chi2: 1.15
[9/1000] vesicle_0008.dat
 radius: 450.0A, thickness: 38.0A,        background: 0.027 (fitted), chi2: 1.00
[10/1000] vesicle_0009.dat
 radius: 4

In [8]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_25/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_25.csv',
          koeff = 0.9975, 
          radius_pd=0.28)

[1/1000] vesicle_0000.dat
 radius: 446.8A, thickness: 47.8A,        background: 0.021 (fitted), chi2: 1.10
[2/1000] vesicle_0001.dat
 radius: 444.9A, thickness: 39.8A,        background: 0.046 (fitted), chi2: 0.74
[3/1000] vesicle_0002.dat
 radius: 447.7A, thickness: 33.6A,        background: 0.023 (fitted), chi2: 0.97
[4/1000] vesicle_0003.dat
 radius: 449.3A, thickness: 40.6A,        background: 0.025 (fitted), chi2: 1.22
[5/1000] vesicle_0004.dat
 radius: 444.1A, thickness: 41.0A,        background: 0.035 (fitted), chi2: 1.77
[6/1000] vesicle_0005.dat
 radius: 447.6A, thickness: 43.3A,        background: 0.024 (fitted), chi2: 1.17
[7/1000] vesicle_0006.dat
 radius: 450.0A, thickness: 43.9A,        background: 0.026 (fitted), chi2: 0.68
[8/1000] vesicle_0007.dat
 radius: 447.6A, thickness: 33.3A,        background: 0.033 (fitted), chi2: 0.52
[9/1000] vesicle_0008.dat
 radius: 450.0A, thickness: 46.5A,        background: 0.029 (fitted), chi2: 1.53
[10/1000] vesicle_0009.dat
 radius: 4

In [9]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_26/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_26.csv',
          koeff = 0.9975, 
          radius_pd=0.28)

[1/1000] vesicle_0000.dat
 radius: 389.5A, thickness: 49.8A,        background: 0.035 (fitted), chi2: 0.44
[2/1000] vesicle_0001.dat
 radius: 352.8A, thickness: 33.4A,        background: 0.026 (fitted), chi2: 0.54
[3/1000] vesicle_0002.dat
 radius: 336.5A, thickness: 39.7A,        background: 0.015 (fitted), chi2: 1.64
[4/1000] vesicle_0003.dat
 radius: 337.0A, thickness: 37.8A,        background: 0.014 (fitted), chi2: 1.99
[5/1000] vesicle_0004.dat
 radius: 311.9A, thickness: 36.6A,        background: 0.023 (fitted), chi2: 0.79
[6/1000] vesicle_0005.dat
 radius: 339.3A, thickness: 30.9A,        background: 0.033 (fitted), chi2: 0.57
[7/1000] vesicle_0006.dat
 radius: 337.9A, thickness: 30.7A,        background: 0.031 (fitted), chi2: 0.57
[8/1000] vesicle_0007.dat
 radius: 350.4A, thickness: 47.4A,        background: 0.026 (fitted), chi2: 0.65
[9/1000] vesicle_0008.dat
 radius: 339.2A, thickness: 40.7A,        background: 0.020 (fitted), chi2: 0.44
[10/1000] vesicle_0009.dat
 radius: 3

In [10]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_27/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_27.csv',
          koeff = 0.9975, 
          radius_pd=0.29)

[1/1000] vesicle_0000.dat
 radius: 446.8A, thickness: 42.0A,        background: 0.025 (fitted), chi2: 1.17
[2/1000] vesicle_0001.dat
 radius: 446.1A, thickness: 30.8A,        background: 0.029 (fitted), chi2: 0.98
[3/1000] vesicle_0002.dat
 radius: 433.6A, thickness: 38.1A,        background: 0.031 (fitted), chi2: 0.90
[4/1000] vesicle_0003.dat
 radius: 427.4A, thickness: 32.9A,        background: 0.032 (fitted), chi2: 0.55
[5/1000] vesicle_0004.dat
 radius: 445.4A, thickness: 30.8A,        background: 0.017 (fitted), chi2: 3.12
[6/1000] vesicle_0005.dat
 radius: 445.7A, thickness: 33.0A,        background: 0.035 (fitted), chi2: 0.60
[7/1000] vesicle_0006.dat
 radius: 444.3A, thickness: 40.3A,        background: 0.021 (fitted), chi2: 3.17
[8/1000] vesicle_0007.dat
 radius: 446.7A, thickness: 40.9A,        background: 0.012 (fitted), chi2: 3.29
[9/1000] vesicle_0008.dat
 radius: 443.2A, thickness: 38.6A,        background: 0.042 (fitted), chi2: 0.81
[10/1000] vesicle_0009.dat
 radius: 4

In [11]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_28/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_28.csv',
          koeff = 0.9975, 
          radius_pd=0.31)

[1/1000] vesicle_0000.dat
 radius: 413.3A, thickness: 30.4A,        background: 0.024 (fitted), chi2: 0.55
[2/1000] vesicle_0001.dat
 radius: 387.3A, thickness: 33.4A,        background: 0.021 (fitted), chi2: 1.54
[3/1000] vesicle_0002.dat
 radius: 450.0A, thickness: 46.4A,        background: 0.027 (fitted), chi2: 1.15
[4/1000] vesicle_0003.dat
 radius: 438.6A, thickness: 32.5A,        background: 0.026 (fitted), chi2: 0.68
[5/1000] vesicle_0004.dat
 radius: 428.1A, thickness: 45.0A,        background: 0.020 (fitted), chi2: 0.61
[6/1000] vesicle_0005.dat
 radius: 426.4A, thickness: 41.6A,        background: 0.009 (fitted), chi2: 2.25
[7/1000] vesicle_0006.dat
 radius: 418.5A, thickness: 44.1A,        background: 0.026 (fitted), chi2: 1.03
[8/1000] vesicle_0007.dat
 radius: 420.7A, thickness: 43.6A,        background: 0.018 (fitted), chi2: 0.74
[9/1000] vesicle_0008.dat
 radius: 415.4A, thickness: 41.8A,        background: 0.028 (fitted), chi2: 0.64
[10/1000] vesicle_0009.dat
 radius: 3

In [12]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_29/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_29.csv',
          koeff = 0.9975, 
          radius_pd=0.32)

[1/1000] vesicle_0000.dat
 radius: 432.3A, thickness: 40.6A,        background: 0.033 (fitted), chi2: 0.56
[2/1000] vesicle_0001.dat
 radius: 430.1A, thickness: 45.3A,        background: 0.025 (fitted), chi2: 0.58
[3/1000] vesicle_0002.dat
 radius: 432.6A, thickness: 38.0A,        background: 0.028 (fitted), chi2: 1.17
[4/1000] vesicle_0003.dat
 radius: 434.7A, thickness: 45.5A,        background: 0.020 (fitted), chi2: 2.26
[5/1000] vesicle_0004.dat
 radius: 431.9A, thickness: 43.9A,        background: 0.034 (fitted), chi2: 0.74
[6/1000] vesicle_0005.dat
 radius: 432.7A, thickness: 45.4A,        background: 0.040 (fitted), chi2: 1.12
[7/1000] vesicle_0006.dat
 radius: 434.5A, thickness: 34.6A,        background: 0.015 (fitted), chi2: 6.54
[8/1000] vesicle_0007.dat
 radius: 433.8A, thickness: 36.6A,        background: 0.036 (fitted), chi2: 0.72
[9/1000] vesicle_0008.dat
 radius: 435.5A, thickness: 32.7A,        background: 0.035 (fitted), chi2: 1.02
[10/1000] vesicle_0009.dat
 radius: 4

In [13]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_30/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_30.csv',
          koeff = 0.9965, 
          radius_pd=0.33)

[1/1000] vesicle_0000.dat
 radius: 429.1A, thickness: 38.5A,        background: 0.034 (fitted), chi2: 0.91
[2/1000] vesicle_0001.dat
 radius: 430.9A, thickness: 34.3A,        background: 0.034 (fitted), chi2: 1.31
[3/1000] vesicle_0002.dat
 radius: 439.3A, thickness: 49.1A,        background: 0.027 (fitted), chi2: 0.32
[4/1000] vesicle_0003.dat
 radius: 431.6A, thickness: 46.5A,        background: 0.037 (fitted), chi2: 0.54
[5/1000] vesicle_0004.dat
 radius: 429.4A, thickness: 39.1A,        background: 0.034 (fitted), chi2: 0.84
[6/1000] vesicle_0005.dat
 radius: 434.8A, thickness: 42.7A,        background: 0.014 (fitted), chi2: 3.09
[7/1000] vesicle_0006.dat
 radius: 438.0A, thickness: 38.9A,        background: 0.038 (fitted), chi2: 0.68
[8/1000] vesicle_0007.dat
 radius: 438.5A, thickness: 36.3A,        background: 0.027 (fitted), chi2: 1.83
[9/1000] vesicle_0008.dat
 radius: 438.8A, thickness: 37.5A,        background: 0.028 (fitted), chi2: 0.80
[10/1000] vesicle_0009.dat
 radius: 4

In [14]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_31/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_31.csv',
          koeff = 0.9965, 
          radius_pd=0.34)

[1/1000] vesicle_0000.dat
 radius: 437.9A, thickness: 47.4A,        background: 0.026 (fitted), chi2: 1.23
[2/1000] vesicle_0001.dat
 radius: 428.8A, thickness: 30.3A,        background: 0.034 (fitted), chi2: 1.31
[3/1000] vesicle_0002.dat
 radius: 425.9A, thickness: 38.8A,        background: 0.036 (fitted), chi2: 1.05
[4/1000] vesicle_0003.dat
 radius: 425.4A, thickness: 38.5A,        background: 0.040 (fitted), chi2: 0.73
[5/1000] vesicle_0004.dat
 radius: 422.4A, thickness: 44.6A,        background: 0.029 (fitted), chi2: 2.34
[6/1000] vesicle_0005.dat
 radius: 426.0A, thickness: 38.1A,        background: 0.019 (fitted), chi2: 2.28
[7/1000] vesicle_0006.dat
 radius: 427.8A, thickness: 32.4A,        background: 0.031 (fitted), chi2: 0.99
[8/1000] vesicle_0007.dat
 radius: 424.8A, thickness: 43.1A,        background: 0.019 (fitted), chi2: 2.48
[9/1000] vesicle_0008.dat
 radius: 426.1A, thickness: 40.6A,        background: 0.020 (fitted), chi2: 1.05
[10/1000] vesicle_0009.dat
 radius: 4

In [15]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_32/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_32.csv',
          koeff = 0.9965, 
          radius_pd=0.35)

[1/1000] vesicle_0000.dat
 radius: 426.8A, thickness: 40.4A,        background: 0.019 (fitted), chi2: 3.78
[2/1000] vesicle_0001.dat
 radius: 428.5A, thickness: 44.2A,        background: 0.029 (fitted), chi2: 0.95
[3/1000] vesicle_0002.dat
 radius: 433.2A, thickness: 30.2A,        background: 0.024 (fitted), chi2: 1.00
[4/1000] vesicle_0003.dat
 radius: 429.9A, thickness: 38.4A,        background: 0.039 (fitted), chi2: 0.93
[5/1000] vesicle_0004.dat
 radius: 429.5A, thickness: 39.5A,        background: 0.037 (fitted), chi2: 0.63
[6/1000] vesicle_0005.dat
 radius: 428.3A, thickness: 40.9A,        background: 0.012 (fitted), chi2: 2.91
[7/1000] vesicle_0006.dat
 radius: 429.6A, thickness: 39.7A,        background: 0.024 (fitted), chi2: 0.85
[8/1000] vesicle_0007.dat
 radius: 426.1A, thickness: 46.9A,        background: 0.026 (fitted), chi2: 0.92
[9/1000] vesicle_0008.dat
 radius: 421.2A, thickness: 40.5A,        background: 0.032 (fitted), chi2: 1.38
[10/1000] vesicle_0009.dat
 radius: 4

In [16]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_33/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_33.csv',
          koeff = 0.9965, 
          radius_pd=0.35)

[1/1000] vesicle_0000.dat
 radius: 422.4A, thickness: 47.1A,        background: 0.025 (fitted), chi2: 4.51
[2/1000] vesicle_0001.dat
 radius: 422.3A, thickness: 38.7A,        background: 0.029 (fitted), chi2: 2.20
[3/1000] vesicle_0002.dat
 radius: 419.3A, thickness: 44.6A,        background: 0.024 (fitted), chi2: 1.43
[4/1000] vesicle_0003.dat
 radius: 421.5A, thickness: 37.2A,        background: 0.023 (fitted), chi2: 1.90
[5/1000] vesicle_0004.dat
 radius: 423.5A, thickness: 32.7A,        background: 0.015 (fitted), chi2: 3.84
[6/1000] vesicle_0005.dat
 radius: 421.5A, thickness: 38.7A,        background: 0.016 (fitted), chi2: 1.63
[7/1000] vesicle_0006.dat
 radius: 420.7A, thickness: 39.7A,        background: 0.021 (fitted), chi2: 1.36
[8/1000] vesicle_0007.dat
 radius: 419.3A, thickness: 49.0A,        background: 0.022 (fitted), chi2: 1.32
[9/1000] vesicle_0008.dat
 radius: 419.9A, thickness: 48.2A,        background: 0.033 (fitted), chi2: 0.98
[10/1000] vesicle_0009.dat
 radius: 4

In [17]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_34/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_34.csv',
          koeff = 0.9965, 
          radius_pd=0.36)

[1/1000] vesicle_0000.dat
 radius: 434.3A, thickness: 39.7A,        background: 0.039 (fitted), chi2: 1.19
[2/1000] vesicle_0001.dat
 radius: 435.2A, thickness: 38.8A,        background: 0.029 (fitted), chi2: 1.30
[3/1000] vesicle_0002.dat
 radius: 436.6A, thickness: 37.1A,        background: 0.023 (fitted), chi2: 1.92
[4/1000] vesicle_0003.dat
 radius: 433.8A, thickness: 42.7A,        background: 0.020 (fitted), chi2: 2.57
[5/1000] vesicle_0004.dat
 radius: 430.1A, thickness: 31.4A,        background: 0.016 (fitted), chi2: 0.93
[6/1000] vesicle_0005.dat
 radius: 427.5A, thickness: 35.1A,        background: 0.022 (fitted), chi2: 2.08
[7/1000] vesicle_0006.dat
 radius: 423.8A, thickness: 48.7A,        background: 0.026 (fitted), chi2: 1.27
[8/1000] vesicle_0007.dat
 radius: 421.3A, thickness: 45.4A,        background: 0.026 (fitted), chi2: 1.27
[9/1000] vesicle_0008.dat
 radius: 416.7A, thickness: 42.9A,        background: 0.024 (fitted), chi2: 1.41
[10/1000] vesicle_0009.dat
 radius: 4

In [18]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_35/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_35.csv',
          koeff = 0.9965, 
          radius_pd=0.37)

[1/1000] vesicle_0000.dat
 radius: 429.2A, thickness: 43.9A,        background: 0.032 (fitted), chi2: 1.09
[2/1000] vesicle_0001.dat
 radius: 431.5A, thickness: 37.5A,        background: 0.029 (fitted), chi2: 0.94
[3/1000] vesicle_0002.dat
 radius: 431.9A, thickness: 35.3A,        background: 0.027 (fitted), chi2: 1.29
[4/1000] vesicle_0003.dat
 radius: 430.0A, thickness: 41.1A,        background: 0.020 (fitted), chi2: 0.97
[5/1000] vesicle_0004.dat
 radius: 429.8A, thickness: 47.2A,        background: 0.026 (fitted), chi2: 1.85
[6/1000] vesicle_0005.dat
 radius: 433.0A, thickness: 48.6A,        background: 0.031 (fitted), chi2: 2.23
[7/1000] vesicle_0006.dat
 radius: 433.1A, thickness: 32.4A,        background: 0.047 (fitted), chi2: 1.49
[8/1000] vesicle_0007.dat
 radius: 430.8A, thickness: 41.2A,        background: 0.031 (fitted), chi2: 1.52
[9/1000] vesicle_0008.dat
 radius: 436.8A, thickness: 43.5A,        background: 0.029 (fitted), chi2: 1.70
[10/1000] vesicle_0009.dat
 radius: 4

In [19]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_36/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_36.csv',
          koeff = 0.9965, 
          radius_pd=0.38)

[1/1000] vesicle_0000.dat
 radius: 428.8A, thickness: 31.2A,        background: 0.027 (fitted), chi2: 1.13
[2/1000] vesicle_0001.dat
 radius: 425.2A, thickness: 42.1A,        background: 0.016 (fitted), chi2: 4.09
[3/1000] vesicle_0002.dat
 radius: 424.1A, thickness: 44.1A,        background: 0.032 (fitted), chi2: 1.04
[4/1000] vesicle_0003.dat
 radius: 427.9A, thickness: 35.3A,        background: 0.022 (fitted), chi2: 2.44
[5/1000] vesicle_0004.dat
 radius: 427.6A, thickness: 38.2A,        background: 0.033 (fitted), chi2: 0.92
[6/1000] vesicle_0005.dat
 radius: 422.9A, thickness: 49.5A,        background: 0.025 (fitted), chi2: 1.83
[7/1000] vesicle_0006.dat
 radius: 419.7A, thickness: 33.8A,        background: 0.022 (fitted), chi2: 3.38
[8/1000] vesicle_0007.dat
 radius: 421.6A, thickness: 29.9A,        background: 0.029 (fitted), chi2: 1.97
[9/1000] vesicle_0008.dat
 radius: 416.5A, thickness: 42.1A,        background: 0.021 (fitted), chi2: 1.42
[10/1000] vesicle_0009.dat
 radius: 4

In [20]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_37/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_37.csv',
          koeff = 0.9965, 
          radius_pd=0.39)

[1/1000] vesicle_0000.dat
 radius: 425.3A, thickness: 33.3A,        background: 0.021 (fitted), chi2: 1.42
[2/1000] vesicle_0001.dat
 radius: 419.4A, thickness: 46.2A,        background: 0.035 (fitted), chi2: 0.95
[3/1000] vesicle_0002.dat
 radius: 421.8A, thickness: 41.4A,        background: 0.027 (fitted), chi2: 1.06
[4/1000] vesicle_0003.dat
 radius: 420.0A, thickness: 46.0A,        background: 0.033 (fitted), chi2: 2.32
[5/1000] vesicle_0004.dat
 radius: 424.6A, thickness: 34.1A,        background: 0.034 (fitted), chi2: 1.49
[6/1000] vesicle_0005.dat
 radius: 419.3A, thickness: 48.9A,        background: 0.015 (fitted), chi2: 3.58
[7/1000] vesicle_0006.dat
 radius: 418.7A, thickness: 48.7A,        background: 0.024 (fitted), chi2: 1.31
[8/1000] vesicle_0007.dat
 radius: 418.9A, thickness: 47.6A,        background: 0.027 (fitted), chi2: 1.84
[9/1000] vesicle_0008.dat
 radius: 423.4A, thickness: 36.3A,        background: 0.026 (fitted), chi2: 1.36
[10/1000] vesicle_0009.dat
 radius: 4

In [21]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_38/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_38.csv',
          koeff = 0.9965, 
          radius_pd=0.4)

[1/1000] vesicle_0000.dat
 radius: 423.1A, thickness: 46.5A,        background: 0.032 (fitted), chi2: 1.69
[2/1000] vesicle_0001.dat
 radius: 427.5A, thickness: 35.4A,        background: 0.033 (fitted), chi2: 1.04
[3/1000] vesicle_0002.dat
 radius: 423.5A, thickness: 44.7A,        background: 0.023 (fitted), chi2: 1.02
[4/1000] vesicle_0003.dat
 radius: 423.2A, thickness: 44.6A,        background: 0.038 (fitted), chi2: 0.94
[5/1000] vesicle_0004.dat
 radius: 428.1A, thickness: 33.6A,        background: 0.028 (fitted), chi2: 0.97
[6/1000] vesicle_0005.dat
 radius: 428.8A, thickness: 31.8A,        background: 0.037 (fitted), chi2: 0.75
[7/1000] vesicle_0006.dat
 radius: 429.1A, thickness: 32.3A,        background: 0.015 (fitted), chi2: 3.48
[8/1000] vesicle_0007.dat
 radius: 422.9A, thickness: 46.4A,        background: 0.032 (fitted), chi2: 1.16
[9/1000] vesicle_0008.dat
 radius: 429.2A, thickness: 30.7A,        background: 0.038 (fitted), chi2: 0.92
[10/1000] vesicle_0009.dat
 radius: 4

In [22]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_39/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_39.csv',
          koeff = 0.9965, 
          radius_pd=0.42)

[1/1000] vesicle_0000.dat
 radius: 434.2A, thickness: 33.5A,        background: 0.018 (fitted), chi2: 3.57
[2/1000] vesicle_0001.dat
 radius: 432.1A, thickness: 49.9A,        background: 0.027 (fitted), chi2: 1.67
[3/1000] vesicle_0002.dat
 radius: 428.7A, thickness: 37.7A,        background: 0.037 (fitted), chi2: 2.13
[4/1000] vesicle_0003.dat
 radius: 429.2A, thickness: 35.6A,        background: 0.026 (fitted), chi2: 0.97
[5/1000] vesicle_0004.dat
 radius: 423.4A, thickness: 49.7A,        background: 0.027 (fitted), chi2: 1.23
[6/1000] vesicle_0005.dat
 radius: 426.7A, thickness: 42.2A,        background: 0.028 (fitted), chi2: 0.93
[7/1000] vesicle_0006.dat
 radius: 428.6A, thickness: 35.9A,        background: 0.038 (fitted), chi2: 1.16
[8/1000] vesicle_0007.dat
 radius: 426.4A, thickness: 41.4A,        background: 0.019 (fitted), chi2: 5.51
[9/1000] vesicle_0008.dat
 radius: 433.7A, thickness: 38.7A,        background: 0.020 (fitted), chi2: 1.55
[10/1000] vesicle_0009.dat
 radius: 4

In [23]:
batch_fit(r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\synthetic_vesicles_polydisperse_p_0_40/*.dat',
          r'C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview\fitted_synthetic_vesicles_polydisperse_p_0_40.csv',
          koeff = 0.9955, 
          radius_pd=0.43)

[1/1000] vesicle_0000.dat
 radius: 425.8A, thickness: 49.4A,        background: 0.029 (fitted), chi2: 2.07
[2/1000] vesicle_0001.dat
 radius: 424.3A, thickness: 42.9A,        background: 0.017 (fitted), chi2: 2.51
[3/1000] vesicle_0002.dat
 radius: 424.9A, thickness: 48.9A,        background: 0.027 (fitted), chi2: 1.83
[4/1000] vesicle_0003.dat
 radius: 434.6A, thickness: 30.7A,        background: 0.021 (fitted), chi2: 0.97
[5/1000] vesicle_0004.dat
 radius: 429.4A, thickness: 41.3A,        background: 0.046 (fitted), chi2: 1.45
[6/1000] vesicle_0005.dat
 radius: 425.4A, thickness: 48.4A,        background: 0.049 (fitted), chi2: 0.77
[7/1000] vesicle_0006.dat
 radius: 431.8A, thickness: 35.2A,        background: 0.032 (fitted), chi2: 1.03
[8/1000] vesicle_0007.dat
 radius: 432.5A, thickness: 34.3A,        background: 0.027 (fitted), chi2: 1.30
[9/1000] vesicle_0008.dat
 radius: 433.2A, thickness: 32.2A,        background: 0.029 (fitted), chi2: 0.79
[10/1000] vesicle_0009.dat
 radius: 4

Соединим все пары датасетов - полученные фитированные результаты с соответствующими синтетическими малоугловыми кривыми.

In [24]:
base_path_1 = r"C:\Users\ksa18\YandexDisk\ML\yumo\synthetic_datasets"
base_path_2 = r"C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview"

processed_datasets = []

for i in range(20, 41):

    df1 = pd.read_csv(f"{base_path_1}/synthetic_dataset_p_0_{i}.csv")
    df2 = pd.read_csv(f"{base_path_2}/fitted_synthetic_vesicles_polydisperse_p_0_{i}.csv")

    # Проверим, что количество строк совпадает
    assert len(df1) == len(df2), f"Размеры датасетов не совпадают для i={i}"
    
    df_merged = pd.concat([df1, df2], axis=1)
    processed_datasets.append(df_merged)

Выведем один из получившихся датасетов для проверки.

In [25]:
display(processed_datasets[0].head())
display(processed_datasets[0].info())

,q_angstrom,I,errors_I,errors_q,errors_I_rel,errors_q_rel,thickness,radius,polydispersity,filename,radius_fitted,thickness_fitted,scale,background,chi2
0,[0.00870964 0.00906512 0.00943512 0.00982022 0...,[2.95440974e+02 3.38235120e+02 3.58586398e+02 ...,[3.52948954e+01 3.97464459e+01 3.59767550e+01 ...,[0.00109112 0.00113808 0.00114038 0.00118734 0...,[11.94651336 11.75112917 10.03293912 9.824550...,[12.52776624 12.5544417 12.08655719 12.090743...,49.869801,399.987108,0.2,vesicle_0000.dat,418.747109,49.187062,0.531946,0.026923,2.015209
1,[0.00870964 0.00906512 0.00943512 0.00982022 0...,[2.91212969e+02 2.77453380e+02 3.10281695e+02 ...,[4.69643387e+01 4.43713734e+01 4.35373941e+01 ...,[0.00106137 0.00114438 0.00120354 0.00118526 0...,[16.1271453 15.9923708 14.03157029 13.181006...,[12.18618926 12.62402118 12.75592619 12.069613...,40.340601,399.910489,0.2,vesicle_0001.dat,419.960512,39.875596,0.660623,0.022592,1.282382
2,[0.00870964 0.00906512 0.00943512 0.00982022 0...,[2.99533541e+02 3.09127055e+02 2.86647795e+02 ...,[3.57339549e+01 3.99073765e+01 3.31065881e+01 ...,[0.00106886 0.00109628 0.00121808 0.00116627 0...,[11.9298676 12.90970034 11.54957015 11.093189...,[12.2721741 12.09340402 12.91004695 11.876202...,40.679360,399.728271,0.2,vesicle_0002.dat,420.099032,39.912231,0.657804,0.005000,3.390163
3,[0.00870964 0.00906512 0.00943512 0.00982022 0...,[3.15507620e+02 3.12045591e+02 3.38627985e+02 ...,[4.12013899e+01 3.81043684e+01 3.77578461e+01 ...,[0.00107002 0.00117797 0.00115897 0.00122501 0...,[13.05876222 12.21115423 11.15024387 10.454424...,[12.2855276 12.99456769 12.28361346 12.474414...,41.308001,399.381731,0.2,vesicle_0003.dat,416.129838,41.499613,0.640041,0.029617,1.038466
4,[0.00870964 0.00906512 0.00943512 0.00982022 0...,[2.46127341e+02 2.90527575e+02 2.80732554e+02 ...,[2.96325318e+01 3.65359231e+01 3.35418757e+01 ...,[0.00105167 0.00112201 0.0012027 0.00122558 0...,[12.03951242 12.57571613 11.94798223 11.267773...,[12.0747869 12.37725813 12.74706727 12.480200...,47.937970,398.813139,0.2,vesicle_0004.dat,411.868015,47.713568,0.560823,0.020099,2.580986


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 15 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   q_angstrom        1000 non-null   object 
 1   I                 1000 non-null   object 
 2   errors_I          1000 non-null   object 
 3   errors_q          1000 non-null   object 
 4   errors_I_rel      1000 non-null   object 
 5   errors_q_rel      1000 non-null   object 
 6   thickness         1000 non-null   float64
 7   radius            1000 non-null   float64
 8   polydispersity    1000 non-null   float64
 9   filename          1000 non-null   object 
 10  radius_fitted     1000 non-null   float64
 11  thickness_fitted  1000 non-null   float64
 12  scale             1000 non-null   float64
 13  background        1000 non-null   float64
 14  chi2              1000 non-null   float64
dtypes: float64(8), object(7)
memory usage: 117.3+ KB


None

Для каждого параметра полидисперсности везикул подготовим единый датасет, содержащий кривые малоуглового рассеяния и полученные в данном разделе работы значения целевого признака thickness, соответствующего своей кривой, для которой он был рассчитан из процедуры фитирования.

In [26]:
output_dir = r"C:\Users\ksa18\YandexDisk\ML\yumo\for_sasview"

os.makedirs(output_dir, exist_ok=True)

for i, dataset in enumerate(processed_datasets, 20):
    filename = f"fitted_for_train_synthetic_vesicles_polydisperse_p_0_{i}.csv"
    filepath = os.path.join(output_dir, filename)
    dataset.to_csv(filepath, index=False)